In [14]:
import math
import os

## Preparación del corpus de emails

In [15]:
!git clone https://github.com/pachocamacho1990/datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


In [ ]:
! unzip datasets/email/plaintext/corpus1.zip

In [ ]:
os.listdir('corpus1/spam')

In [18]:
data = []
clases = []
#lectura de spam data
for file in os.listdir('corpus1/spam'):
  with open('corpus1/spam/'+file, encoding='latin-1') as f:
    data.append(f.read())
    clases.append('spam')
#lectura de ham data
for file in os.listdir('corpus1/ham'):
  with open('corpus1/ham/'+file, encoding='latin-1') as f:
    data.append(f.read())
    clases.append('ham')
len(data)

5172

In [19]:
len(data), len(clases)

(5172, 5172)

## Construcción de modelo Naive Bayes

### Tokenizador de Spacy

* Documentación: https://spacy.io/api/tokenizer
* ¿Cómo funciona el tokenizador? https://spacy.io/usage/linguistic-features#how-tokenizer-works

In [20]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

In [21]:
print([t.text for t in tokenizer(data[0])])

['Subject:', 'drug', 'turns', 'a', 'normal', 'guys', 'into', 'studs', '!', '\n', 'cialls', 'delivered', 'discreetly', 'to', 'your', 'door', 'in', 'an', 'unmarked', 'envelope', '\n', 'lasts', '8', 'x', 'times', 'longer', 'than', 'vlagra', ',', 'take', 'one', 'friday', ',', 'still', 'working', 'on', 'sunday', '!', '\n', 'no', 'one', 'needs', 'to', 'know', '!', '\n', 'buy', 'generic', '(', 'exactly', 'the', 'same', 'drug', ',', 'but', 'not', 'with', 'the', 'big', 'company', 'prices', '.', '.', ')', 'to', 'save', '70', '%', '!', '!', '\n', 'check', 'it', 'out', 'here', '!', '\n', 'fireball', 'abcdzhongguo', 'johnson', 'daddy', 'front', '242', 'gretchen', 'memory', '\n', 'josie', 'buttonshorizon', 'buffy', 'preston', '\n', 'josh', 'informix', 'diane', '\n']


### Clase principal para el algoritmo

Recuerda que la clase más probable viene dada por (en espacio de cómputo logarítmico): 


$$\hat{c} = {\arg \max}_{(c)}\log{P(c)}
 +\sum_{i=1}^n
\log{ P(f_i \vert c)}
$$

Donde, para evitar casos atípicos, usaremos el suavizado de Laplace así:

$$
P(f_i \vert c) = \frac{C(f_i, c)+1}{C(c) + \vert V \vert}
$$

siendo $\vert V \vert$ la longitud del vocabulario de nuestro conjunto de entrenamiento. 

In [22]:
import numpy as np

class NaiveBayesClassifier():
  nlp = English()
  tokenizer = Tokenizer(nlp.vocab)
  
  def tokenize(self, doc):
    return  [t.text.lower() for t in tokenizer(doc)]

  def word_counts(self, words):
    '''hace el conteo de las palabras'''
    wordCount = {}
    for w in words: 
      if w in wordCount.keys():
        wordCount[w] += 1
      else:
        wordCount[w] = 1
    return wordCount

  def fit(self, data, clases):
    '''Calcula todas las probabilidades'''
    n = len(data)
    self.unique_clases = set(clases)
    self.vocab = set()
    self.classCount = {} #C(c)
    self.log_classPriorProb = {} #P(c)
    self.wordConditionalCounts = {} #C(w|c)
    #conteos de clases
    for c in clases:
      if c in self.classCount.keys():
        self.classCount[c] += 1
      else:
        self.classCount[c] = 1
    # calculo de P(c)
    for c in self.classCount.keys():
      self.log_classPriorProb[c] = math.log(self.classCount[c]/n)
      self.wordConditionalCounts[c] = {}
    # calculo de C(w|c)
    for text, c in zip(data, clases):
      counts = self.word_counts(self.tokenize(text))
      for word, count in counts.items():
        # Agregamos palabra a vocab
        if word not in self.vocab:
          self.vocab.add(word)
        if word not in self.wordConditionalCounts[c]:
          self.wordConditionalCounts[c][word] = 0.0
        self.wordConditionalCounts[c][word] += count

  def predict(self, data):
    results = []
    for text in data:
      words = set(self.tokenize(text))
      scoreProb = {}
      for word in words: 
        if word not in self.vocab: continue #ignoramos palabras nuevas
        #suavizado Laplaciano para P(w|c)
        for c in self.unique_clases:
          log_wordClassProb = math.log(
              (self.wordConditionalCounts[c].get(word, 0.0)+1)/(self.classCount[c]+len(self.vocab)))
          scoreProb[c] = scoreProb.get(c, self.log_classPriorProb[c]) + log_wordClassProb
      # obtenemos argumento de maxima probabilidad
      arg_maxprob = np.argmax(np.array(list(scoreProb.values())))
      results.append(list(scoreProb.keys())[arg_maxprob])
    return results


### Utilidades de Scikit Learn
* `train_test_split`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

* `accuracy_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

* `precision_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

* `recall_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
data_train, data_test, clases_train, clases_test = train_test_split(data, clases, test_size=0.10, random_state=42)

In [24]:
classifier = NaiveBayesClassifier()
classifier.fit(data_train, clases_train)

In [25]:
clases_predict = classifier.predict(data_test)

In [26]:
accuracy_score(clases_test, clases_predict)

0.8397683397683398

In [27]:
precision_score(clases_test, clases_predict, average=None, zero_division=1)

array([0.81390135, 1.        ])

In [30]:
# de todo lo que predije que era ham solo el 0.81% es ham | todo lo que predije como spam fue stpam

In [28]:
recall_score(clases_test, clases_predict, average=None, zero_division=1)

array([1.        , 0.46451613])

In [32]:
# De todo lo que en el dataset realmente es ham capture el 100% | 
# de todo lo que en el dataset es spam logre capturar solo el 51% 